# Tensorflow with Estimators

In [1]:
import pandas as pd

In [2]:
'''from sklearn import datasets
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)'''
df = pd.read_csv('dataset/iris.csv')

In [3]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [5]:
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']

In [6]:
X = df.drop('target',axis=1)
y = df['target'].apply(int) # set target column type as int

## Train Test Split

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Estimators

In [9]:
import tensorflow as tf

/Users/mk194903/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
#Feature Columns

In [11]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [12]:
feat_cols = []
for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [13]:
feat_cols 

[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [14]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=5, shuffle=True) #epoch is times

In [15]:
#Deep Neural Network Classifier or Multilayer Perceptron (MLP)
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=3,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/rf/5ly7zckd3y3fh75zc5llgsr80000gn/T/tmpr9hhblzr', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x113497208>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
classifier.train(input_fn=input_func,steps=100)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/rf/5ly7zckd3y3fh75zc5llgsr80000gn/T/tmpr9hhblzr/model.ckpt.
INFO:tensorflow:loss = 11.029797, step = 1
INFO:tensorflow:Saving checkpoints for 53 into /var/folders/rf/5ly7zckd3y3fh75zc5llgsr80000gn/T/tmpr9hhblzr/model.ckpt.
INFO:tensorflow:Loss for final step: 2.0737433.


## Model Evaluation

In [17]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [18]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/rf/5ly7zckd3y3fh75zc5llgsr80000gn/T/tmpr9hhblzr/model.ckpt-53
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [19]:
note_predictions[0]

{'logits': array([-2.9057057 , -0.56561327, -1.7303183 ], dtype=float32),
 'probabilities': array([0.06839199, 0.7100591 , 0.22154886], dtype=float32),
 'class_ids': array([1]),
 'classes': array([b'1'], dtype=object)}

In [20]:
final_preds =[]
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [21]:
from sklearn.metrics import classification_report, confusion_matrix

In [22]:
print(confusion_matrix(y_test, final_preds))

[[15  0  0]
 [ 0 14  1]
 [ 0  2 13]]


In [23]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        15
          1       0.88      0.93      0.90        15
          2       0.93      0.87      0.90        15

avg / total       0.93      0.93      0.93        45

